In [ ]:
# Hello!
# So I want to look at crop diversity against the factor scores
# And livestock diversity
# And break this down by market orientation

# Confirms use of PAF and PCA largely equivalent, but PCA is easier for interpretation
# regression coefficients are larger because loadings are larger so scores are larger (see scales)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
from scipy.stats import pearsonr
import statsmodels.api as sm

In [ ]:
def normalise(dataframe):
    for i in dataframe.columns:
        (m,s) = (dataframe[i].mean(), dataframe[i].std())
        dataframe.loc[:,i]=dataframe.loc[:,i]-m
        dataframe.loc[:,i]=np.divide(dataframe.loc[:,i],s)
        print ("for column ",str(i), ", the mean was ",m," and the standard deviation was ",s)
    return dataframe

In [ ]:
indicators = pd.read_csv("3PCAonImputedValues_output",encoding = "ISO-8859-1")

In [ ]:
analysis = None
possible_analyses = ["PCA","PAF"]
analysis = input("PCA or PAF? ")
while (analysis in possible_analyses) == False:
    analysis = input("You have to choose PCA or PAF.\n PCA or PAF? ")

In [ ]:
# name the important columns, and normalise them so that the unit is standard deviations from the mean
if analysis == 'PCA':
    factors_temp = normalise(indicators[['Food Security PCA','Diet Quality PCA','Money PCA']])
    factors_temp.rename(columns={'Food Security PCA':'Food Security Factor','Diet Quality PCA':'Diet Quality Factor',
                                 'Money PCA':'Money Factor'}, inplace=True)
    
else:
    factors_temp = normalise(indicators[['Food Security PAF','Diet Quality PAF','Money PAF']])
    factors_temp.rename(columns={'Food Security PAF':'Food Security Factor','Diet Quality PAF':'Diet Quality Factor',
                                 'Money PAF':'Money Factor'}, inplace=True)

indicators = pd.concat([indicators, factors_temp],axis=1)

In [ ]:
# Check how correlated the factors are with each other
indicators[['Food Security Factor','Diet Quality Factor','Money Factor']].corr()
# seems reasonable

In [ ]:
# indicators.info()

In [ ]:
# Check the distribution of the households with food insecurity missing
indicators[['Country', 'FI missing', 'HHid']].groupby(['Country', 'FI missing']).count()

In [ ]:
# only Burkina Faso and Mali have some FI there and some missing, the others are split entirely by country
# suggests that this is a slightly dodgy measure...

In [ ]:
# Get ready to group by village
indicators['Coun Reg Vill']=indicators['Country']+indicators['Region']+indicators['Village'].astype(str)
indicators.head()

In [ ]:
def ols_params(x,y):
    testmod = sm.OLS(y,sm.add_constant(x))
    tes = testmod.fit()
    a = tes.params[1]
    s = tes.bse[1]
    rsq = tes.rsquared
    return (a,s, rsq)
#ols_params(indic['CropDiv'], indic['Diet Quality Factor'])

In [ ]:
# Look at crop and livestock diversity against the factors.
# first, crop diversity by household;
# the correlation coefficients are not very big,
# but this does appear to vary by country.

# get rid of the small number of missing values
indic=indicators[indicators['CropDiv']<10000]
indil=indicators[indicators['LivestockDiv']<10000]

fig = plt.figure(figsize = (10,10))
ax = fig.subplots(3)
corrs = ['CropDiv','Money Factor','Food Security Factor','Diet Quality Factor']
sns.regplot('CropDiv', 'Money Factor', data=indicators, ax=ax[0])
#sns.lmplot('CropDiv', 'Money Factor', data=indicators, hue = 'Country')
sns.regplot('CropDiv', 'Food Security Factor', data=indicators, ax=ax[1])
#sns.lmplot('CropDiv', 'Food Security Factor', data=indicators, hue = 'Country')
sns.regplot('CropDiv', 'Diet Quality Factor', data=indicators, ax=ax[2])
#sns.lmplot('CropDiv', 'Diet Quality Factor', data=indicators, hue = 'Country')
if analysis == "PAF":
    fig.savefig('4 PAF 01 Crop diversity against factor scores by household')
else:
    fig.savefig('4 PCA 01 Crop diversity against factor scores by household')
for i in range (3):
    r=pearsonr(indic[corrs[0]],indic[corrs[1+i]])
    (a,s, Rsq)=ols_params(indic['CropDiv'], indic[corrs[1+i]])
    ax[i].text(20,-2.5-i*(2-i)*2, 'Corr coeff: '+str(round(r[0],3))+' Sig level: '+str(round(r[1],5))+' n = '+str(len(indic))+
              '\nReg coeff: '+str(round(a,3))+' Std err: '+str(round(s,3))+' Rsq: '+str(round(Rsq,3)))


In [ ]:
data = np.array([[1,1],[2,6],[3,8],[7,30],[2,5]])
df = pd.DataFrame(data, columns = ['Independent', 'Dependent'])
df

In [ ]:
testmod = sm.OLS(df['Dependent'],sm.add_constant(df['Independent']) )

tes = testmod.fit()

print(tes.summary())

In [ ]:
tes.rsquared

In [ ]:
tes.bse

In [ ]:
first_grouping_c = indic[['CropDiv','Food Security Factor',
       'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill']].groupby('Coun Reg Vill').mean()
first_grouping_l = indil[['LivestockDiv','Food Security Factor',
       'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill']].groupby('Coun Reg Vill').mean()

In [ ]:
# Repeat the scatter graphs by village, not by household
# The correlation coefficients are now much larger, and the shapes can be seen on the scatter graphs
fig = plt.figure(figsize = (10,12))
axes = fig.subplots(3, gridspec_kw={'hspace':.4})
for i in range(3):
    axes[i].scatter(x=first_grouping_c['CropDiv'], y=first_grouping_c[first_grouping_c.columns[1+i]], alpha = .6)
    axes[i].set_ylabel(str(first_grouping_c.columns[1+i]))
    axes[i].set_xlabel('Average crop diversity in village')
    rp = pearsonr(first_grouping_c['CropDiv'], first_grouping_c[first_grouping_c.columns[1+i]])
    r = round(rp[0], 3)
    p=round(rp[1],5)
    (a, s, Rsq) = ols_params(first_grouping_c['CropDiv'], first_grouping_c[first_grouping_c.columns[1+i]])
    axes[i].text(8.2,1+0.5*i*abs(1.667-i),'Corr coef: '+str(r)+' Sig level: '+str(p)+
                 ' (n = '+str(len(first_grouping_c))+')'+
              '\nReg coeff: '+str(round(a,3))+' Std err: '+str(round(s,3))+' Rsq: '+str(round(Rsq,3)))
if analysis=='PCA':
    fig.savefig('4 PCA 02 Crop diversity against factor scores by village')
else:
    fig.savefig('4 PAF 02 Crop diversity against factor scores by village')

In [ ]:
# for livestock diversity, all households
# again, the coefficients are small but significant
fig = plt.figure(figsize = (10,10))
ax = fig.subplots(3)
factors=['Food Security Factor','Diet Quality Factor','Money Factor']
for i in range(3):
    plot = sns.regplot('LivestockDiv', factors[i], data=indil, ax=ax[i])
    rp = pearsonr(indil['LivestockDiv'], indil[factors[i]])
    r = round(rp[0], 3)
    p=round(rp[1],5)
    (a, s, Rsq) = ols_params(indil['LivestockDiv'], indil[factors[i]])
    plot.text(5.7,-3.2 + i*abs(1.667-i),'Pearson correlation coefficient: '+str(r)+'\nSignificance level: '+str(p)+
                 ' (n = '+str(len(indil))+')'+
              '\nReg coeff: '+str(round(a,3))+' Std err: '+str(round(s,3))+' Rsq: '+str(round(Rsq,3)))
if analysis=='PCA':    
    fig.savefig('4 PCA 03 Livestock diversity against factor scores by household')
else:
    fig.savefig('4 PAF 03 Livestock diversity against factor scores by household')

In [ ]:
# this time the correlation coefficients are small even at the village level

fig = plt.figure(figsize = (10,12))
axes = fig.subplots(3, gridspec_kw={'hspace':.4})
for i in range(3):
    axes[i].scatter(x=first_grouping_l['LivestockDiv'], y=first_grouping_l[first_grouping_l.columns[1+i]], alpha = .6)
    axes[i].set_ylabel(str(first_grouping_l.columns[1+i]))
    axes[i].set_xlabel('Average livestock diversity in village')
    rp = pearsonr(first_grouping_l['LivestockDiv'], first_grouping_l[first_grouping_l.columns[1+i]])
    r = round(rp[0], 3)
    p=round(rp[1],5)
    (a, s, Rsq) = ols_params(first_grouping_l['LivestockDiv'], first_grouping_l[first_grouping_l.columns[1+i]])
    axes[i].text(4.35,.8+0.4*i*abs(1.667-i),'Pearson correlation coefficient: '+str(r)+'\nSignificance level: '+str(p)+
                 ' (n = '+str(len(first_grouping_l))+')'+
              '\nReg coeff: '+str(round(a,3))+' Std err: '+str(round(s,3))+' Rsq: '+str(round(Rsq,3)))
if analysis=='PAF':
    fig.savefig('4 PAF 04 Livestock diversity against factor scores by village')
else:
    fig.savefig('4 PCA 04 Livestock diversity against factor scores by village')

In [ ]:
len(indicators['Country'].unique())

In [ ]:
fig = plt.figure(figsize = (12,12))
#axes = fig.subplots(3, gridspec_kw={'hspace':.4, 'wspace':2})
colours = ('red','green','blue','yellow','orange','black','purple','brown','grey','hotpink','teal')
for i in range(3):
    axes[i]=fig.add_axes([.07,0.667-0.333*i+.05,.55,0.26])
    for j in range(11):
        subset = indil[indil['Country']==indil['Country'].unique()[j]]
        grouping = subset[['CropDiv', 'LivestockDiv','Food Security Factor',
                               'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill']].groupby('Coun Reg Vill').mean()
        axes[i].scatter(x=grouping['LivestockDiv'], y=grouping[grouping.columns[2+i]], alpha = .6, color=colours[j])
        axes[i].set_ylabel(str(grouping.columns[2+i]))
        axes[i].set_xlabel('Average livestock diversity in village')
        rp = pearsonr(grouping['LivestockDiv'], grouping[grouping.columns[2+i]])
        r = round(rp[0], 3)
        p = round(rp[1],5)
        n = len(grouping)
        (a, s, Rsq) = ols_params(grouping['LivestockDiv'], grouping[grouping.columns[2+i]])
#        axes[i].text(7.5, 1.8+0.2*i-0.5*j*(10+i+i*(2-i))*0.1+0.1*i*(2-i), indicators['Country'].unique()[j]+'; r = '+str(r)+'; p = '+
#                     str(p)+'; n = '+str(n)+';\n   reg = '+str(round(a,3))+'; se = '+str(round(s,3))+
 #                    '; Rsq = '+str(round(Rsq,3)), color=colours[j])
        axes[i].text(7.5,1.5+0.75*i*abs(1.667-i)-0.08*i*abs(1.667-i)*j-0.4*j, indicators['Country'].unique()[j]+': n = '+str(n)+';\n   reg = '+str(round(a,3))+'; se = '+str(round(s,3))+
                     '; Rsq = '+str(round(Rsq,3)), color=colours[j])
if analysis=='PCA':
    fig.savefig('4 PCA 05 Livestock diversity against factor scores by village by country')
else:
    fig.savefig('4 PAF 05 Livestock diversity against factor scores by village by country')



In [ ]:
fig = plt.figure(figsize = (14,12))
#axes = fig.subplots(3, gridspec_kw={'hspace':.4})
colours = ('red','green','blue','yellow','orange','black','purple','brown','grey','hotpink','teal')
for i in range(3):
    axes[i]=fig.add_axes([.07,0.72-0.333*i+.05,.6,0.23])
    for j in range(11):
        subset = indic[indic['Country']==indic['Country'].unique()[j]]
        grouping = subset[['CropDiv', 'LivestockDiv','Food Security Factor',
                               'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill']].groupby('Coun Reg Vill').mean()
        axes[i].scatter(x=grouping['CropDiv'], y=grouping[grouping.columns[2+i]], alpha = .6, color=colours[j])
        axes[i].set_ylabel(str(grouping.columns[2+i]))
        axes[i].set_xlabel('Average crop diversity in village')
        rp = pearsonr(grouping['CropDiv'], grouping[grouping.columns[2+i]])
        r = round(rp[0], 3)
        p = round(rp[1],5)
        n = len(grouping)
        (a,s,Rsq)=ols_params(grouping['CropDiv'], grouping[grouping.columns[2+i]])
#        axes[i].text(14, 1.8+0.2*i-0.6*j*(10+i+i*(2-i))*0.1+0.1*i*(2-i), indicators['Country'].unique()[j]+'; r = '+str(r)+'; p = '+
 #                    str(p)+' (n = '+str(n)+')\n   reg = '+str(round(a,3))+'; se = '+str(round(s,3))+
  #                   '; Rsq = '+str(round(Rsq,3)), color=colours[j])
        axes[i].text(14, 1.2+i*abs(1.667-i)-0.1*i*abs(1.667-i)*j-0.45*j, indicators['Country'].unique()[j]+': n = '+str(n)+'\n   reg = '+str(round(a,3))+'; se = '+str(round(s,3))+
                     '; Rsq = '+str(round(Rsq,3)), color=colours[j])

if analysis == "PCA":
    fig.savefig('4 PCA 06 Crop diversity against factor scores by village by country')
else:
    fig.savefig('4 PAF 06 Crop diversity against factor scores by village by country')

In [ ]:
# next, I want to split it up by market orientation
# I had 4 market orientation quartiles before.
prepc=indic[indic['FAMarketOrientation']<1.1]
prepl=indil[indil['FAMarketOrientation']<1.1]

second_grouping_c = prepc[['CropDiv', 'LivestockDiv','Food Security Factor',
       'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill', 'FAMarketOrientation']].groupby('Coun Reg Vill').mean()
second_grouping_l = prepl[['CropDiv', 'LivestockDiv','Food Security Factor',
       'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill', 'FAMarketOrientation']].groupby('Coun Reg Vill').mean()

In [ ]:
second_grouping_c['MAQuartile']=0
for j in range(4):
 #   print(second_grouping[['FAMarketOrientation','MAQuartile']][11:20])
    q = second_grouping_c['FAMarketOrientation'].quantile(0.25*(j+1))
  #  print(q)
    second_grouping_c['MAQuartile']=np.where(second_grouping_c['FAMarketOrientation']>q, second_grouping_c['MAQuartile']+1, second_grouping_c['MAQuartile'])
    
second_grouping_l['MAQuartile']=0
for j in range(4):
 #   print(second_grouping[['FAMarketOrientation','MAQuartile']][11:20])
    q = second_grouping_l['FAMarketOrientation'].quantile(0.25*(j+1))
  #  print(q)
    second_grouping_l['MAQuartile']=np.where(second_grouping_l['FAMarketOrientation']>q, second_grouping_l['MAQuartile']+1, second_grouping_l['MAQuartile'])

In [ ]:
second_grouping_l.info()

In [ ]:
fig = plt.figure(figsize = (10,12))
#axes = fig.subplots(3, gridspec_kw={'hspace':.4})
colours = ('red','orange','green','blue')
for i in range(3):
    axes[i]=fig.add_axes([.07,0.667-0.333*i+.05,.55,0.22])
    for j in range(4):
        grouping = second_grouping_l[second_grouping_l['MAQuartile']==j]
        axes[i].scatter(x=grouping['LivestockDiv'], y=grouping[grouping.columns[2+i]], alpha = .6, color=colours[j])
        axes[i].set_ylabel(str(grouping.columns[2+i]))
        axes[i].set_xlabel('Average livestock diversity in village')
        rp = pearsonr(grouping['LivestockDiv'], grouping[grouping.columns[2+i]])
        r = round(rp[0],3)
        p = round(rp[1],5)
        n = len(grouping)
        (a, s, Rsq ) = ols_params(grouping['LivestockDiv'], grouping[grouping.columns[2+i]])
#        axes[i].text(7.5, 1.8-0.8*j, 'MA Quartile '+str(j+1)+'; r = '+str(r)+'; p = '+str(p)+' (n = '+str(n)+')\n   reg = '
 #                    +str(round(a,3))+'; se = '+str(round(s,3))+
  #                   '; Rsq = '+str(round(Rsq,3)), color=colours[j])
        axes[i].text(7.5, 1.4-0.8*j, 'MA Quartile '+str(j+1)+': n = '+str(n)+'\n   reg = '
                     +str(round(a,3))+'; se = '+str(round(s,3))+
                     '; Rsq = '+str(round(Rsq,3)), color=colours[j])
if analysis =='PAF':
    fig.savefig('4 PAF 07 Livestock diversity against factor scores by village by MA quartile')
else:
    fig.savefig('4 PCA 07 Livestock diversity against factor scores by village by MA quartile')

In [ ]:
fig = plt.figure(figsize = (14,12))
#axes = fig.subplots(3, gridspec_kw={'hspace':.4})
colours = ('red','orange','green','blue')
for i in range(3):
    axes[i]=fig.add_axes([.07,0.667-0.333*i+.05,.63,0.22])
    for j in range(4):
        grouping = second_grouping_c[second_grouping_c['MAQuartile']==j]
        axes[i].scatter(x=grouping['CropDiv'], y=grouping[grouping.columns[2+i]], alpha = .6, color=colours[j])
        axes[i].set_ylabel(str(grouping.columns[2+i]))
        axes[i].set_xlabel('Average crop diversity in village')
        rp = pearsonr(grouping['CropDiv'], grouping[grouping.columns[2+i]])
        r = round(rp[0], 3)
        p = round(rp[1],5)
        n = len(grouping)
        (a,s,Rsq) = ols_params(grouping['CropDiv'], grouping[grouping.columns[2+i]])
#        axes[i].text(14, 1.8-0.8*j, 'MA Quartile '+str(j+1)+'; r = '+str(r)+'; p = '+str(p)+' (n = '+str(n)+')\n   reg = '
 #                    +str(round(a,3))+'; se = '+str(round(s,3))+
  #                   '; Rsq = '+str(round(Rsq,3)), color=colours[j])
        axes[i].text(14, 1.4-0.8*j, 'MA Quartile '+str(j+1)+': n = '+str(n)+'\n   reg = '
                     +str(round(a,3))+'; se = '+str(round(s,3))+
                     '; Rsq = '+str(round(Rsq,3)), color=colours[j])
if analysis == 'PCA':
    fig.savefig('4 PCA 08 Crop diversity against factor scores by village by MA quartile')
else:
    fig.savefig('4 PAF 08 Crop diversity against factor scores by village by MA quartile')

In [ ]:
# make a column for the quartiles
prepc['MAQuartile']=0
for j in range(4):
 #   print(second_grouping[['FAMarketOrientation','MAQuartile']][11:20])
    q = prepc['FAMarketOrientation'].quantile(0.25*(j+1))
  #  print(q)
    prepc['MAQuartile']=np.where(prepc['FAMarketOrientation']>q, prepc['MAQuartile']+1, prepc['MAQuartile'])
    
prepl['MAQuartile']=0
for j in range(4):
 #   print(second_grouping[['FAMarketOrientation','MAQuartile']][11:20])
    q = prepl['FAMarketOrientation'].quantile(0.25*(j+1))
  #  print(q)
    prepl['MAQuartile']=np.where(prepl['FAMarketOrientation']>q, prepl['MAQuartile']+1, prepl['MAQuartile'])

In [ ]:
factors

In [ ]:
fig = plt.figure(figsize = (14,12))
#axes = fig.subplots(3, gridspec_kw={'hspace':.4})
colours = ('red','orange','green','blue')
for i in range(3):
    for j in range(4):
        axes=fig.add_axes([.07+0.24*j,0.667-0.333*i+.05,0.19,0.22])
        grouping = prepl[prepl['MAQuartile']==j]
        axes.scatter(x=grouping['LivestockDiv'], y=grouping[factors[i]], alpha = .1, color=colours[j])
        axes.set_ylabel(str(factors[i]))
        axes.set_xlabel('Livestock diversity')
        rp = pearsonr(grouping['LivestockDiv'], grouping[factors[i]])
        r = round(rp[0],3)
        p = round(rp[1],5)
        n = len(grouping)
        (a,s,Rsq) = ols_params(grouping['LivestockDiv'], grouping[factors[i]])
        axes.set_title('MA Quartile '+str(j+1)+': reg = '
                     +str(round(a,3))+'; se = '+str(round(s,3))+
                     ';\n Rsq = '+str(round(Rsq,3))+' (n = '+str(n)+')', color=colours[j], size=10)
if analysis == 'PCA':
    fig.savefig('4 PCA 09 Livestock diversity against factor scores by household by MA quartile')
else:
    fig.savefig('4 PAF 09 Livestock diversity against factor scores by household by MA quartile')

In [ ]:
fig = plt.figure(figsize = (16,12))
#axes = fig.subplots(3, gridspec_kw={'hspace':.4})
colours = ('red','orange','green','blue')
for i in range(3):
    for j in range(4):
        axes=fig.add_axes([.07+0.24*j,0.667-0.333*i+.05,.19,0.22])
        grouping = prepc[prepc['MAQuartile']==j]
        axes.scatter(x=grouping['CropDiv'], y=grouping[factors[i]], alpha = .1, color=colours[j])
        axes.set_ylabel(str(factors[i]))
        axes.set_xlabel('Crop diversity')
        rp = pearsonr(grouping['CropDiv'], grouping[factors[i]])
        r = round(rp[0],3)
        p = round(rp[1],5)
        n = len(grouping)
        (a,s,Rsq)=ols_params(grouping['CropDiv'], grouping[factors[i]])
        axes.set_title('MA Quartile '+str(j+1)+': reg = '
                     +str(round(a,3))+'; se = '+str(round(s,3))+
                     ';\n Rsq = '+str(round(Rsq,3))+' (n = '+str(n)+')', color=colours[j], size=10)
if analysis == 'PCA':
    fig.savefig('4 PCA 10 Crop diversity against factor scores by household by MA quartile')
else:
    fig.savefig('4 PAF 10 Crop diversity against factor scores by household by MA quartile')

In [ ]:
if analysis == 'PCA':
    sns.jointplot(x='CropDiv', y='FAMarketOrientation',data=indicators, alpha = .1).savefig('4 PCA 11 cropdiv by market orientation')
    sns.jointplot(x='LivestockDiv', y='FAMarketOrientation',data=indicators, alpha=.1).savefig('4 PCA 12 livestockdiv by market orientation')
    sns.jointplot(x='FAMarketOrientation', y='Money Factor',data=indicators, alpha=.1).savefig('4 PCA 13 market orientation by money')
else:
    sns.jointplot(x='CropDiv', y='FAMarketOrientation',data=indicators, alpha = .1).savefig('4 PAF 11 cropdiv by market orientation')
    sns.jointplot(x='LivestockDiv', y='FAMarketOrientation',data=indicators, alpha=.1).savefig('4 PAF 12 livestockdiv by market orientation')
    sns.jointplot(x='FAMarketOrientation', y='Money Factor',data=indicators, alpha=.1).savefig('4 PAF 13 market orientation by money')

In [ ]:
if analysis == 'PCA':
    sns.jointplot(x='CropDiv', y='FAMarketOrientation',data=second_grouping_c, alpha = .5).savefig('4 PCA 14 cropdiv by market orientation')
    sns.jointplot(x='LivestockDiv', y='FAMarketOrientation',data=second_grouping_l, alpha=.5).savefig('4 PCA 15 livestockdiv by market orientation')
    sns.jointplot(x='FAMarketOrientation', y='Money Factor',data=second_grouping_l, alpha=.5).savefig('4 PCA 16 market orientation by money')
else:
    sns.jointplot(x='CropDiv', y='FAMarketOrientation',data=second_grouping_c, alpha = .5).savefig('4 PCA 14 cropdiv by market orientation')
    sns.jointplot(x='LivestockDiv', y='FAMarketOrientation',data=second_grouping_l, alpha=.5).savefig('4 PCA 15 livestockdiv by market orientation')
    sns.jointplot(x='FAMarketOrientation', y='Money Factor',data=second_grouping_l, alpha=.5).savefig('4 PCA 16 market orientation by money')

In [ ]:
# and does Market Orientation affect crop/livestock diversity at the village level?
sns.jointplot(x='FAMarketOrientation', y='CropDiv', data=second_grouping_c, alpha = .4).savefig('4 Market orientation and crop div by village')
sns.jointplot(x='FAMarketOrientation', y='LivestockDiv', data=second_grouping_l, alpha = .4).savefig('4 Market orientation and livestock div by village')